In [1]:
from pathlib import Path

# Paths to your files (edit this to your folder)
# data_dir = Path("/../en-ta.txt/")
en_path = "/Users/niteeshkumar/Documents/document_level_nmt/en-ta.txt/pmindia.en-ta.en"
ta_path = "/Users/niteeshkumar/Documents/document_level_nmt/en-ta.txt/pmindia.en-ta.ta"

# 1. Read files
with open(en_path, "r", encoding="utf-8") as f_en:
    en_lines = [line.strip() for line in f_en]

with open(ta_path, "r", encoding="utf-8") as f_ta:
    ta_lines = [line.strip() for line in f_ta]

assert len(en_lines) == len(ta_lines), "Line counts do not match!"

# 2. Build parallel sentence pairs
parallel_pairs = [
    {"en": src, "ta": tgt}
    for src, tgt in zip(en_lines, ta_lines)
    if src and tgt  # drop empty lines
]

print(f"Total pairs: {len(parallel_pairs)}")
print(parallel_pairs[0])


Total pairs: 10577
{'en': 'Prime Minister Narendra Modi met Her Majesty Queen Maxima of the Kingdom of the Netherlands today.', 'ta': 'சுகாதாரம் மற்றும் குடும்பநலத் துறையின் கீழ் உள்ள தன்னாட்சி அமைப்புகளை சீரமைக்கும் நடவடிக்கையில், அமைச்சகங்களுக்கு இடையேயான ஆலோசனைகள் நடைபெறும் மற்றும் இந்த அமைப்புகளுக்கான ஏற்கனவே உள்ள சட்டதிட்டங்கள் மறுஆய்வு செய்யப்படும்.'}


In [2]:
from pathlib import Path

tsv_path = "/Users/niteeshkumar/Documents/document_level_nmt/pmindia.v1.ta-en.tsv" # update this

with open(tsv_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        print(repr(line))
        if i == 9:   # first 10 lines
            break


'The rationalization of Autonomous Bodies under Department of Health & Family Welfare will involve inter-ministerial consultations and review of existing bye laws of these bodies.\tசுகாதாரம் மற்றும் குடும்பநலத் துறையின் கீழ் உள்ள தன்னாட்சி அமைப்புகளை சீரமைக்கும் நடவடிக்கையில், அமைச்சகங்களுக்கு இடையேயான ஆலோசனைகள் நடைபெறும் மற்றும் இந்த அமைப்புகளுக்கான ஏற்கனவே உள்ள சட்டதிட்டங்கள் மறுஆய்வு செய்யப்படும்.\n'
'The time frame for implementation is one year,\tஇதனை செயல்படுத்துவதற்கான காலக்கெடு ஓராண்டு.\n'
'Since the DoHFW provides funds to the hospitals, the grants can be given from the Department to the hospital directly.\tமருத்துவமனைகளுக்கான நிதியை சுகாதாரம் மற்றும் குடும்ப நலத் துறை வழங்கி வருவதால், நிதியுதவியையும் மருத்துமனைகளுக்கு துறையே நேரில் வழங்க முடியும்.\n'
'RAN functions can, therefore, be vested in DoHFW.\tதேசிய ஆரோக்கிய நிதி அமைப்பின் செயல்பாடுகள், சுகாதாரம் மற்றும் குடும்பநலத் துறையிடம் இருக்கும்.\n'
'Managing Committee of RAN Society will meet to dissolve the Autonomous Body (A

In [3]:
pairs = []
with open(tsv_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.rstrip("\n")
        if not line:
            continue

        parts = line.split("\t")
        if len(parts) != 2:
            continue  # skip any abnormal lines

        en, ta = parts
        en, ta = en.strip(), ta.strip()
        if en and ta:
            pairs.append({"en": en, "ta": ta})

print("Total pairs:", len(pairs))
print(pairs[0])

Total pairs: 39526
{'en': 'The rationalization of Autonomous Bodies under Department of Health & Family Welfare will involve inter-ministerial consultations and review of existing bye laws of these bodies.', 'ta': 'சுகாதாரம் மற்றும் குடும்பநலத் துறையின் கீழ் உள்ள தன்னாட்சி அமைப்புகளை சீரமைக்கும் நடவடிக்கையில், அமைச்சகங்களுக்கு இடையேயான ஆலோசனைகள் நடைபெறும் மற்றும் இந்த அமைப்புகளுக்கான ஏற்கனவே உள்ள சட்டதிட்டங்கள் மறுஆய்வு செய்யப்படும்.'}


In [4]:
# !pip install -U sentence-transformers

In [5]:
# from sentence_transformers import SentenceTransformer, util
# import torch

# model = SentenceTransformer("sentence-transformers/LaBSE")

# batch_size = 64
# aligned_pairs = []

# for start in range(0, len(pairs), batch_size):
#     end = start + batch_size
#     en_batch = [p["en"] for p in pairs[start:end]]
#     ta_batch = [p["ta"] for p in pairs[start:end]]

#     en_emb = model.encode(en_batch, convert_to_tensor=True)
#     ta_emb = model.encode(ta_batch, convert_to_tensor=True)

#     cos_scores = util.cos_sim(en_emb, ta_emb).diagonal()  # only batch diag

#     for pair, score in zip(pairs[start:end], cos_scores):
#         if score.item() >= 0.7:
#             aligned_pairs.append({
#                 "en": pair["en"],
#                 "ta": pair["ta"],
#                 "score": float(score),
#             })

# print("Original:", len(pairs), "Aligned:", len(aligned_pairs))


In [6]:
# shape(aligned_pairs)

In [7]:
DOC_LEN = 8  # sentences per pseudo-document

documents = []
for i in range(0, len(pairs), DOC_LEN):
    chunk = pairs[i:i+DOC_LEN]
    if len(chunk) < 2:
        break
    documents.append(chunk)

print("Num documents:", len(documents))
print("First doc len:", len(documents[0]))

Num documents: 4941
First doc len: 8


In [8]:
documents[0]

[{'en': 'The rationalization of Autonomous Bodies under Department of Health & Family Welfare will involve inter-ministerial consultations and review of existing bye laws of these bodies.',
  'ta': 'சுகாதாரம் மற்றும் குடும்பநலத் துறையின் கீழ் உள்ள தன்னாட்சி அமைப்புகளை சீரமைக்கும் நடவடிக்கையில், அமைச்சகங்களுக்கு இடையேயான ஆலோசனைகள் நடைபெறும் மற்றும் இந்த அமைப்புகளுக்கான ஏற்கனவே உள்ள சட்டதிட்டங்கள் மறுஆய்வு செய்யப்படும்.'},
 {'en': 'The time frame for implementation is one year,',
  'ta': 'இதனை செயல்படுத்துவதற்கான காலக்கெடு ஓராண்டு.'},
 {'en': 'Since the DoHFW provides funds to the hospitals, the grants can be given from the Department to the hospital directly.',
  'ta': 'மருத்துவமனைகளுக்கான நிதியை சுகாதாரம் மற்றும் குடும்ப நலத் துறை வழங்கி வருவதால், நிதியுதவியையும் மருத்துமனைகளுக்கு துறையே நேரில் வழங்க முடியும்.'},
 {'en': 'RAN functions can, therefore, be vested in DoHFW.',
  'ta': 'தேசிய ஆரோக்கிய நிதி அமைப்பின் செயல்பாடுகள், சுகாதாரம் மற்றும் குடும்பநலத் துறையிடம் இருக்கும்.'},
 {'en':

In [9]:
len(pairs)

39526

In [10]:
import pandas as pd

df = pd.DataFrame(pairs)
df

,en,ta
0,The rationalization of Autonomous Bodies under...,சுகாதாரம் மற்றும் குடும்பநலத் துறையின் கீழ் உள...
1,"The time frame for implementation is one year,",இதனை செயல்படுத்துவதற்கான காலக்கெடு ஓராண்டு.
2,Since the DoHFW provides funds to the hospital...,மருத்துவமனைகளுக்கான நிதியை சுகாதாரம் மற்றும் க...
3,"RAN functions can, therefore, be vested in DoHFW.","தேசிய ஆரோக்கிய நிதி அமைப்பின் செயல்பாடுகள், சு..."
4,Managing Committee of RAN Society will meet to...,"சங்கங்கள் பதிவுச் சட்டம், 1860-ல் உள்ள வழிமுறை..."
...,...,...
39521,We will also get to see its direct benefit dur...,இரு நாடுகளையும் சேர்ந்த தொழில் துறையினருடன் இன...
39522,I am confident that our dialogue and the agree...,இன்றைய நமது பேச்சுவார்த்தைகள் மற்றும் கையெழுத்...
39523,"With these words, I once again warmly welcome ...","இந்தக் கருத்துகளுடன், பிரதமர் அபே-வையும், அவரு..."
39524,Ijyo De Gozaimas (That’s all for now),Ijyo De Gozaimas (தற்போது இவ்வளவு தான்)


In [11]:
df=df[:1000]
df.shape

(1000, 2)

In [12]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import math
# import json
# from torch.utils.data import Dataset, DataLoader
# from collections import Counter
# import numpy as np

# class PositionalEncoding(nn.Module):
#     def __init__(self, d_model, max_len=5000):
#         super().__init__()
#         pe = torch.zeros(max_len, d_model)
#         position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
#                            (-math.log(10000.0) / d_model))
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0).transpose(0, 1)
#         self.register_buffer('pe', pe)

#     def forward(self, x):
#         return x + self.pe[:x.size(0), :]

In [13]:
# import math
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# # ---------- Positional Encoding ----------

# class PositionalEncoding(nn.Module):
#     def __init__(self, d_model, max_len=5000, dropout=0.1):
#         super().__init__()
#         self.dropout = nn.Dropout(p=dropout)

#         pe = torch.zeros(max_len, d_model)          # [max_len, d_model]
#         position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, d_model, 2).float()
#                              * (-math.log(10000.0) / d_model))
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0)                        # [1, max_len, d_model]
#         self.register_buffer("pe", pe)

#     def forward(self, x):
#         # x: [batch_size, seq_len, d_model]
#         x = x + self.pe[:, :x.size(1), :]
#         return self.dropout(x)


# # ---------- Multi-Head Attention ----------

# class MultiHeadAttention(nn.Module):
#     def __init__(self, d_model, n_heads, dropout=0.1):
#         super().__init__()
#         assert d_model % n_heads == 0
#         self.d_model = d_model
#         self.n_heads = n_heads
#         self.d_k = d_model // n_heads

#         self.W_q = nn.Linear(d_model, d_model)
#         self.W_k = nn.Linear(d_model, d_model)
#         self.W_v = nn.Linear(d_model, d_model)
#         self.W_o = nn.Linear(d_model, d_model)

#         self.dropout = nn.Dropout(dropout)

#     def forward(self, query, key, value, mask=None):
#         # query/key/value: [batch, seq_len, d_model]
#         batch_size = query.size(0)

#         # Linear projections
#         Q = self.W_q(query)  # [batch, seq, d_model]
#         K = self.W_k(key)
#         V = self.W_v(value)

#         # Split into heads: [batch, heads, seq, d_k]
#         def split_heads(x):
#             return x.view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)

#         Q = split_heads(Q)
#         K = split_heads(K)
#         V = split_heads(V)

#         # Scaled dot-product attention
#         scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
#         # scores: [batch, heads, seq_q, seq_k]

#         if mask is not None:
#             # mask: [batch, 1, 1, seq_k] or [batch, 1, seq_q, seq_k]
#             scores = scores.masked_fill(mask == 0, float("-inf"))

#         attn = F.softmax(scores, dim=-1)
#         attn = self.dropout(attn)

#         context = torch.matmul(attn, V)  # [batch, heads, seq_q, d_k]

#         # Concatenate heads
#         context = context.transpose(1, 2).contiguous().view(
#             batch_size, -1, self.d_model
#         )  # [batch, seq_q, d_model]

#         out = self.W_o(context)
#         return out


# # ---------- Position-wise Feed Forward ----------

# class PositionwiseFeedForward(nn.Module):
#     def __init__(self, d_model, d_ff, dropout=0.1):
#         super().__init__()
#         self.fc1 = nn.Linear(d_model, d_ff)
#         self.fc2 = nn.Linear(d_ff, d_model)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x):
#         return self.fc2(self.dropout(F.relu(self.fc1(x))))


# # ---------- Encoder / Decoder Layers ----------

# class EncoderLayer(nn.Module):
#     def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
#         super().__init__()
#         self.self_attn = MultiHeadAttention(d_model, n_heads, dropout)
#         self.ff = PositionwiseFeedForward(d_model, d_ff, dropout)
#         self.norm1 = nn.LayerNorm(d_model)
#         self.norm2 = nn.LayerNorm(d_model)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x, src_mask=None):
#         # Self-attention
#         attn_out = self.self_attn(x, x, x, src_mask)
#         x = self.norm1(x + self.dropout(attn_out))
#         # Feed-forward
#         ff_out = self.ff(x)
#         x = self.norm2(x + self.dropout(ff_out))
#         return x


# class DecoderLayer(nn.Module):
#     def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
#         super().__init__()
#         self.self_attn = MultiHeadAttention(d_model, n_heads, dropout)
#         self.cross_attn = MultiHeadAttention(d_model, n_heads, dropout)
#         self.ff = PositionwiseFeedForward(d_model, d_ff, dropout)

#         self.norm1 = nn.LayerNorm(d_model)
#         self.norm2 = nn.LayerNorm(d_model)
#         self.norm3 = nn.LayerNorm(d_model)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x, memory, tgt_mask=None, memory_mask=None):
#         # Masked self-attention (decoder)
#         self_attn_out = self.self_attn(x, x, x, tgt_mask)
#         x = self.norm1(x + self.dropout(self_attn_out))
#         # Encoder–decoder attention
#         cross_attn_out = self.cross_attn(x, memory, memory, memory_mask)
#         x = self.norm2(x + self.dropout(cross_attn_out))
#         # Feed-forward
#         ff_out = self.ff(x)
#         x = self.norm3(x + self.dropout(ff_out))
#         return x


# # ---------- Encoder / Decoder Stacks ----------

# class Encoder(nn.Module):
#     def __init__(self, vocab_size, d_model, N, n_heads, d_ff, dropout=0.1, max_len=5000):
#         super().__init__()
#         self.tok_embed = nn.Embedding(vocab_size, d_model)
#         self.pos_embed = PositionalEncoding(d_model, max_len, dropout)
#         self.layers = nn.ModuleList(
#             [EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(N)]
#         )

#     def forward(self, src, src_mask=None):
#         # src: [batch, src_len]
#         x = self.tok_embed(src)   # [batch, src_len, d_model]
#         x = self.pos_embed(x)
#         for layer in self.layers:
#             x = layer(x, src_mask)
#         return x


# class Decoder(nn.Module):
#     def __init__(self, vocab_size, d_model, N, n_heads, d_ff, dropout=0.1, max_len=5000):
#         super().__init__()
#         self.tok_embed = nn.Embedding(vocab_size, d_model)
#         self.pos_embed = PositionalEncoding(d_model, max_len, dropout)
#         self.layers = nn.ModuleList(
#             [DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(N)]
#         )
#         self.fc_out = nn.Linear(d_model, vocab_size)

#     def forward(self, tgt, memory, tgt_mask=None, memory_mask=None):
#         # tgt: [batch, tgt_len]
#         x = self.tok_embed(tgt)
#         x = self.pos_embed(x)
#         for layer in self.layers:
#             x = layer(x, memory, tgt_mask, memory_mask)
#         logits = self.fc_out(x)
#         return logits


# # ---------- Full Transformer ----------

# def generate_square_subsequent_mask(sz, device):
#     # standard causal mask for decoder
#     mask = torch.triu(torch.ones(sz, sz, device=device), diagonal=1).bool()
#     return ~mask  # True where allowed, False where masked


# class Transformer(nn.Module):
#     def __init__(
#         self,
#         src_vocab_size,
#         tgt_vocab_size,
#         d_model=512,
#         N=6,
#         n_heads=8,
#         d_ff=2048,
#         dropout=0.1,
#         max_len=5000,
#     ):
#         super().__init__()
#         self.encoder = Encoder(src_vocab_size, d_model, N, n_heads, d_ff, dropout, max_len)
#         self.decoder = Decoder(tgt_vocab_size, d_model, N, n_heads, d_ff, dropout, max_len)

#     def make_src_mask(self, src, pad_idx):
#         # src: [batch, src_len]
#         mask = (src != pad_idx).unsqueeze(1).unsqueeze(2)
#         # [batch, 1, 1, src_len]
#         return mask

#     def make_tgt_mask(self, tgt, pad_idx):
#         # tgt: [batch, tgt_len]
#         batch_size, tgt_len = tgt.size()
#         pad_mask = (tgt != pad_idx).unsqueeze(1).unsqueeze(2)  # [batch, 1, 1, tgt_len]
#         subseq_mask = generate_square_subsequent_mask(tgt_len, tgt.device)  # [tgt_len, tgt_len]
#         subseq_mask = subseq_mask.unsqueeze(0).unsqueeze(1)  # [1, 1, tgt_len, tgt_len]
#         mask = pad_mask & subseq_mask
#         return mask

#     def forward(self, src, tgt, src_pad_idx, tgt_pad_idx):
#         src_mask = self.make_src_mask(src, src_pad_idx)
#         tgt_mask = self.make_tgt_mask(tgt, tgt_pad_idx)
#         memory = self.encoder(src, src_mask)
#         out = self.decoder(tgt, memory, tgt_mask, src_mask)
#         return out



# # Pseudo-code: adapt to your actual file format.
# def parse_line(line):
#     # Example input:
#     # "enta0The rationalization ...சுகாதாரம் ...1"
#     # You likely need a known separator; if not, pre-clean in a script.
#     # For illustration, assume tab-separated: id \t en \t ta \t idx
#     parts = line.strip().split("\t")
#     en = parts[1]
#     ta = parts[2]
#     return en, ta


In [14]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # After you build vocabularies:
# src_vocab_size = len(src_vocab)
# tgt_vocab_size = len(tgt_vocab)
# pad_idx = 0  # make sure 0 is PAD in both vocabs

# model = Transformer(
#     src_vocab_size=src_vocab_size,
#     tgt_vocab_size=tgt_vocab_size,
#     d_model=512,
#     N=6,
#     n_heads=8,
#     d_ff=2048,
#     dropout=0.1,
#     max_len=256,
# ).to(device)

# criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
# optimizer = torch.optim.Adam(
#     model.parameters(),
#     lr=1e-4,
#     betas=(0.9, 0.98),
#     eps=1e-9,
# )

# for epoch in range(10):
#     model.train()
#     total_loss = 0.0
#     for batch in train_dataloader:
#         src, tgt = batch["src"].to(device), batch["tgt"].to(device)
#         # Teacher forcing: decoder input is tgt[:, :-1], target labels are tgt[:, 1:]
#         tgt_inp = tgt[:, :-1]
#         tgt_out = tgt[:, 1:]

#         logits = model(src, tgt_inp, pad_idx, pad_idx)  # [batch, tgt_len-1, vocab]
#         loss = criterion(
#             logits.reshape(-1, tgt_vocab_size),
#             tgt_out.reshape(-1)
#         )

#         optimizer.zero_grad()
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#         optimizer.step()

#         total_loss += loss.item()

#     print(f"Epoch {epoch+1}: loss = {total_loss / len(train_dataloader):.4f}")


In [15]:
#vanila transformer 

In [16]:
# import math
# from collections import Counter

# import pandas as pd
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader


# # ===============================
# # 1. Vocab building & tokenization
# # ===============================

# PAD_TOKEN = "<pad>"
# SOS_TOKEN = "<sos>"
# EOS_TOKEN = "<eos>"
# UNK_TOKEN = "<unk>"

# special_tokens = [PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, UNK_TOKEN]

# def simple_tokenize(text: str):
#     # Replace with a better tokenizer if you want.
#     return text.strip().split()

# def build_vocab(texts, min_freq=1):
#     counter = Counter()
#     for t in texts:
#         counter.update(simple_tokenize(str(t)))
#     itos = list(special_tokens)
#     for tok, freq in counter.items():
#         if freq >= min_freq and tok not in itos:
#             itos.append(tok)
#     stoi = {tok: idx for idx, tok in enumerate(itos)}
#     return stoi, itos

# # Build vocabularies from dataframe
# src_vocab, src_itos = build_vocab(df["en"].tolist(), min_freq=1)
# tgt_vocab, tgt_itos = build_vocab(df["ta"].tolist(), min_freq=1)

# pad_idx = src_vocab[PAD_TOKEN]
# assert pad_idx == tgt_vocab[PAD_TOKEN]

# def encode_sentence(text, vocab, max_len=256):
#     tokens = [SOS_TOKEN] + simple_tokenize(str(text)) + [EOS_TOKEN]
#     ids = [vocab.get(tok, vocab[UNK_TOKEN]) for tok in tokens]
#     if len(ids) < max_len:
#         ids += [vocab[PAD_TOKEN]] * (max_len - len(ids))
#     else:
#         ids = ids[:max_len]
#     return ids

# max_len = 256

# src_seqs = [encode_sentence(t, src_vocab, max_len=max_len) for t in df["en"]]
# tgt_seqs = [encode_sentence(t, tgt_vocab, max_len=max_len) for t in df["ta"]]

# src_tensor = torch.tensor(src_seqs, dtype=torch.long)
# tgt_tensor = torch.tensor(tgt_seqs, dtype=torch.long)

# # ===============================
# # 2. Dataset & DataLoader
# # ===============================

# class NMTDataset(Dataset):
#     def __init__(self, src_tensor, tgt_tensor):
#         self.src = src_tensor
#         self.tgt = tgt_tensor

#     def __len__(self):
#         return self.src.size(0)

#     def __getitem__(self, idx):
#         return {
#             "src": self.src[idx],
#             "tgt": self.tgt[idx],
#         }

# dataset = NMTDataset(src_tensor, tgt_tensor)

# train_dataloader = DataLoader(
#     dataset,
#     batch_size=64,
#     shuffle=True,
#     drop_last=True,
# )

# # ===============================
# # 3. Transformer components
# # ===============================

# class PositionalEncoding(nn.Module):
#     def __init__(self, d_model, max_len=5000, dropout=0.1):
#         super().__init__()
#         self.dropout = nn.Dropout(p=dropout)

#         pe = torch.zeros(max_len, d_model)          # [max_len, d_model]
#         position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, d_model, 2).float()
#                              * (-math.log(10000.0) / d_model))
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0)                        # [1, max_len, d_model]
#         self.register_buffer("pe", pe)

#     def forward(self, x):
#         # x: [batch_size, seq_len, d_model]
#         x = x + self.pe[:, :x.size(1), :]
#         return self.dropout(x)


# class MultiHeadAttention(nn.Module):
#     def __init__(self, d_model, n_heads, dropout=0.1):
#         super().__init__()
#         assert d_model % n_heads == 0
#         self.d_model = d_model
#         self.n_heads = n_heads
#         self.d_k = d_model // n_heads

#         self.W_q = nn.Linear(d_model, d_model)
#         self.W_k = nn.Linear(d_model, d_model)
#         self.W_v = nn.Linear(d_model, d_model)
#         self.W_o = nn.Linear(d_model, d_model)

#         self.dropout = nn.Dropout(dropout)

#     def forward(self, query, key, value, mask=None):
#         # query/key/value: [batch, seq_len, d_model]
#         batch_size = query.size(0)

#         Q = self.W_q(query)
#         K = self.W_k(key)
#         V = self.W_v(value)

#         def split_heads(x):
#             return x.view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)

#         Q = split_heads(Q)
#         K = split_heads(K)
#         V = split_heads(V)

#         scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
#         # scores: [batch, heads, seq_q, seq_k]

#         if mask is not None:
#             # mask: [batch, 1, 1, seq_k] or [batch, 1, seq_q, seq_k]
#             scores = scores.masked_fill(mask == 0, float("-inf"))

#         attn = F.softmax(scores, dim=-1)
#         attn = self.dropout(attn)

#         context = torch.matmul(attn, V)  # [batch, heads, seq_q, d_k]

#         context = context.transpose(1, 2).contiguous().view(
#             batch_size, -1, self.d_model
#         )  # [batch, seq_q, d_model]

#         out = self.W_o(context)
#         return out


# class PositionwiseFeedForward(nn.Module):
#     def __init__(self, d_model, d_ff, dropout=0.1):
#         super().__init__()
#         self.fc1 = nn.Linear(d_model, d_ff)
#         self.fc2 = nn.Linear(d_ff, d_model)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x):
#         return self.fc2(self.dropout(F.relu(self.fc1(x))))


# class EncoderLayer(nn.Module):
#     def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
#         super().__init__()
#         self.self_attn = MultiHeadAttention(d_model, n_heads, dropout)
#         self.ff = PositionwiseFeedForward(d_model, d_ff, dropout)
#         self.norm1 = nn.LayerNorm(d_model)
#         self.norm2 = nn.LayerNorm(d_model)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x, src_mask=None):
#         attn_out = self.self_attn(x, x, x, src_mask)
#         x = self.norm1(x + self.dropout(attn_out))
#         ff_out = self.ff(x)
#         x = self.norm2(x + self.dropout(ff_out))
#         return x


# class DecoderLayer(nn.Module):
#     def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
#         super().__init__()
#         self.self_attn = MultiHeadAttention(d_model, n_heads, dropout)
#         self.cross_attn = MultiHeadAttention(d_model, n_heads, dropout)
#         self.ff = PositionwiseFeedForward(d_model, d_ff, dropout)

#         self.norm1 = nn.LayerNorm(d_model)
#         self.norm2 = nn.LayerNorm(d_model)
#         self.norm3 = nn.LayerNorm(d_model)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x, memory, tgt_mask=None, memory_mask=None):
#         self_attn_out = self.self_attn(x, x, x, tgt_mask)
#         x = self.norm1(x + self.dropout(self_attn_out))
#         cross_attn_out = self.cross_attn(x, memory, memory, memory_mask)
#         x = self.norm2(x + self.dropout(cross_attn_out))
#         ff_out = self.ff(x)
#         x = self.norm3(x + self.dropout(ff_out))
#         return x


# class Encoder(nn.Module):
#     def __init__(self, vocab_size, d_model, N, n_heads, d_ff, dropout=0.1, max_len=5000):
#         super().__init__()
#         self.tok_embed = nn.Embedding(vocab_size, d_model)
#         self.pos_embed = PositionalEncoding(d_model, max_len, dropout)
#         self.layers = nn.ModuleList(
#             [EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(N)]
#         )

#     def forward(self, src, src_mask=None):
#         x = self.tok_embed(src)   # [batch, src_len, d_model]
#         x = self.pos_embed(x)
#         for layer in self.layers:
#             x = layer(x, src_mask)
#         return x


# class Decoder(nn.Module):
#     def __init__(self, vocab_size, d_model, N, n_heads, d_ff, dropout=0.1, max_len=5000):
#         super().__init__()
#         self.tok_embed = nn.Embedding(vocab_size, d_model)
#         self.pos_embed = PositionalEncoding(d_model, max_len, dropout)
#         self.layers = nn.ModuleList(
#             [DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(N)]
#         )
#         self.fc_out = nn.Linear(d_model, vocab_size)

#     def forward(self, tgt, memory, tgt_mask=None, memory_mask=None):
#         x = self.tok_embed(tgt)
#         x = self.pos_embed(x)
#         for layer in self.layers:
#             x = layer(x, memory, tgt_mask, memory_mask)
#         logits = self.fc_out(x)
#         return logits


# def generate_square_subsequent_mask(sz, device):
#     mask = torch.triu(torch.ones(sz, sz, device=device), diagonal=1).bool()
#     return ~mask  # True where allowed, False where masked


# class Transformer(nn.Module):
#     def __init__(
#         self,
#         src_vocab_size,
#         tgt_vocab_size,
#         d_model=512,
#         N=6,
#         n_heads=8,
#         d_ff=2048,
#         dropout=0.1,
#         max_len=5000,
#     ):
#         super().__init__()
#         self.encoder = Encoder(src_vocab_size, d_model, N, n_heads, d_ff, dropout, max_len)
#         self.decoder = Decoder(tgt_vocab_size, d_model, N, n_heads, d_ff, dropout, max_len)

#     def make_src_mask(self, src, pad_idx):
#         mask = (src != pad_idx).unsqueeze(1).unsqueeze(2)
#         return mask

#     def make_tgt_mask(self, tgt, pad_idx):
#         batch_size, tgt_len = tgt.size()
#         pad_mask = (tgt != pad_idx).unsqueeze(1).unsqueeze(2)  # [batch, 1, 1, tgt_len]
#         subseq_mask = generate_square_subsequent_mask(tgt_len, tgt.device)  # [tgt_len, tgt_len]
#         subseq_mask = subseq_mask.unsqueeze(0).unsqueeze(1)  # [1, 1, tgt_len, tgt_len]
#         mask = pad_mask & subseq_mask
#         return mask

#     def forward(self, src, tgt, src_pad_idx, tgt_pad_idx):
#         src_mask = self.make_src_mask(src, src_pad_idx)
#         tgt_mask = self.make_tgt_mask(tgt, tgt_pad_idx)
#         memory = self.encoder(src, src_mask)
#         out = self.decoder(tgt, memory, tgt_mask, src_mask)
#         return out

# # ===============================
# # 4. Training setup
# # ===============================

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# src_vocab_size = len(src_vocab)
# tgt_vocab_size = len(tgt_vocab)

# model = Transformer(
#     src_vocab_size=src_vocab_size,
#     tgt_vocab_size=tgt_vocab_size,
#     d_model=512,
#     N=6,
#     n_heads=8,
#     d_ff=2048,
#     dropout=0.1,
#     max_len=max_len,
# ).to(device)

# criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
# optimizer = torch.optim.Adam(
#     model.parameters(),
#     lr=1e-4,
#     betas=(0.9, 0.98),
#     eps=1e-9,
# )

# # ===============================
# # 5. Training loop
# # ===============================

# num_epochs = 10

# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0.0

#     for batch in train_dataloader:
#         src = batch["src"].to(device)
#         tgt = batch["tgt"].to(device)

#         # Teacher forcing: decoder input is tgt[:, :-1], labels are tgt[:, 1:]
#         tgt_inp = tgt[:, :-1]
#         tgt_out = tgt[:, 1:]

#         logits = model(src, tgt_inp, pad_idx, pad_idx)  # [batch, tgt_len-1, vocab]

#         loss = criterion(
#             logits.reshape(-1, tgt_vocab_size),
#             tgt_out.reshape(-1),
#         )

#         optimizer.zero_grad()
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#         optimizer.step()

#         total_loss += loss.item()

#     avg_loss = total_loss / len(train_dataloader)
#     print(f"Epoch {epoch+1}/{num_epochs}: loss = {avg_loss:.4f}")


In [17]:
# import pandas as pd
# import torch
# from torch.utils.data import Dataset, DataLoader
# from transformers import MarianMTModel, MarianTokenizer, AdamW

# # =========================
# # 0. Basic setup
# # =========================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # df = pd.read_csv("pmindia_en_ta_39k.csv")  # ensure df['en'], df['ta'] exist

# # Optional: shuffle / small train–val split
# # df = df.sample(frac=1.0, random_state=42).reset_index(drop=True)
# # val_frac = 0.05
# # val_size = int(len(df) * val_frac)
# # df_train = df.iloc[:-val_size]
# # df_val = df.iloc[-val_size:]


# # =========================
# # 1. Load Marian en→hi model
# # =========================
# model_name = "Helsinki-NLP/opus-mt-en-hi"   # pretrained English→Indic
# tokenizer = MarianTokenizer.from_pretrained(model_name)
# model = MarianMTModel.from_pretrained(model_name).to(device)


# # =========================
# # 2. Dataset class
# # =========================
# class EnTaMarianDataset(Dataset):
#     def __init__(self, df, tokenizer, max_length=128):
#         self.en = df["en"].astype(str).tolist()
#         self.ta = df["ta"].astype(str).tolist()
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.en)

#     def __getitem__(self, idx):
#         src_text = self.en[idx]
#         tgt_text = self.ta[idx]

#         # Source: English
#         src_enc = self.tokenizer(
#             src_text,
#             max_length=self.max_length,
#             truncation=True,
#             padding="max_length",
#             return_tensors="pt",
#         )

#         # Target: Tamil
#         with self.tokenizer.as_target_tokenizer():
#             tgt_enc = self.tokenizer(
#                 tgt_text,
#                 max_length=self.max_length,
#                 truncation=True,
#                 padding="max_length",
#                 return_tensors="pt",
#             )

#         input_ids = src_enc["input_ids"].squeeze(0)
#         attention_mask = src_enc["attention_mask"].squeeze(0)
#         labels = tgt_enc["input_ids"].squeeze(0)

#         # Mask pad tokens in labels with -100 so they are ignored by CE loss
#         labels[labels == tokenizer.pad_token_id] = -100

#         return {
#             "input_ids": input_ids,
#             "attention_mask": attention_mask,
#             "labels": labels,
#         }


# train_dataset = EnTaMarianDataset(df_train, tokenizer, max_length=128)
# val_dataset = EnTaMarianDataset(df_val, tokenizer, max_length=128)

# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


# # =========================
# # 3. Optimizer
# # =========================
# optimizer = AdamW(model.parameters(), lr=3e-5)


# # =========================
# # 4. Training + simple validation
# # =========================
# num_epochs = 3

# for epoch in range(num_epochs):
#     # ---- train ----
#     model.train()
#     total_train_loss = 0.0

#     for batch in train_loader:
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
#         labels = batch["labels"].to(device)

#         outputs = model(
#             input_ids=input_ids,
#             attention_mask=attention_mask,
#             labels=labels,
#         )
#         loss = outputs.loss

#         optimizer.zero_grad()
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#         optimizer.step()

#         total_train_loss += loss.item()

#     avg_train_loss = total_train_loss / len(train_loader)

#     # ---- validation (optional) ----
#     model.eval()
#     total_val_loss = 0.0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch["input_ids"].to(device)
#             attention_mask = batch["attention_mask"].to(device)
#             labels = batch["labels"].to(device)

#             outputs = model(
#                 input_ids=input_ids,
#                 attention_mask=attention_mask,
#                 labels=labels,
#             )
#             loss = outputs.loss
#             total_val_loss += loss.item()

#     avg_val_loss = total_val_loss / len(val_loader)
#     print(f"Epoch {epoch+1}/{num_epochs} | train_loss={avg_train_loss:.4f} | val_loss={avg_val_loss:.4f}")


# # =========================
# # 5. Save fine-tuned model
# # =========================
# model.save_pretrained("marian_enhi_to_entamil_finetuned")
# tokenizer.save_pretrained("marian_enhi_to_entamil_finetuned")


In [ ]:
##Marian model 

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import MarianMTModel, MarianTokenizer, AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# df = pd.read_csv("pmindia_en_ta_39k.csv")  # df['en'], df['ta']

# =========================
# 1. Load Marian en→hi model
# =========================
model_name = "Helsinki-NLP/opus-mt-en-hi"   # pretrained English→Indic
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)


# =========================
# 2. Dataset class (full 39k)
# =========================
class EnTaMarianDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.en = df["en"].astype(str).tolist()
        self.ta = df["ta"].astype(str).tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.en)

    def __getitem__(self, idx):
        src_text = self.en[idx]
        tgt_text = self.ta[idx]

        # Source: English
        src_enc = self.tokenizer(
            src_text,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        # Target: Tamil
        with self.tokenizer.as_target_tokenizer():
            tgt_enc = self.tokenizer(
                tgt_text,
                max_length=self.max_length,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

        input_ids = src_enc["input_ids"].squeeze(0)
        attention_mask = src_enc["attention_mask"].squeeze(0)
        labels = tgt_enc["input_ids"].squeeze(0)

        # Mask pad tokens in labels with -100 so they are ignored by CE loss
        labels[labels == tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

dataset = EnTaMarianDataset(df, tokenizer, max_length=128)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)


# =========================
# 3. Optimizer
# =========================
optimizer = AdamW(model.parameters(), lr=3e-5)


# =========================
# 4. Training loop on full 39k
# =========================
num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0.0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} | train_loss={avg_train_loss:.4f}")


# =========================
# 5. Save fine-tuned model
# =========================
model.save_pretrained("marian_enhi_to_entamil_finetuned_full39k")
tokenizer.save_pretrained("marian_enhi_to_entamil_finetuned_full39k")


In [ ]:
##Marian Model with self training 

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import MarianMTModel, MarianTokenizer, AdamW
import copy

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your data (Assuming the CSV is in the same directory)
# df = pd.read_csv("pmindia_en_ta_39k.csv") 

# ==========================================
# 1. Load Initial Model & Tokenizer
# ==========================================
model_name = "Helsinki-NLP/opus-mt-en-hi"  # Base pre-trained model
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)

# ==========================================
# 2. Dataset Class for Supervised Learning
# ==========================================
class EnTaMarianDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.en = df["en"].astype(str).tolist()
        self.ta = df["ta"].astype(str).tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.en)

    def __getitem__(self, idx):
        src_text = self.en[idx]
        tgt_text = self.ta[idx]

        src_enc = self.tokenizer(src_text, max_length=self.max_length, 
                                 truncation=True, padding="max_length", return_tensors="pt")
        
        with self.tokenizer.as_target_tokenizer():
            tgt_enc = self.tokenizer(tgt_text, max_length=self.max_length, 
                                     truncation=True, padding="max_length", return_tensors="pt")

        input_ids = src_enc["input_ids"].squeeze(0)
        attention_mask = src_enc["attention_mask"].squeeze(0)
        labels = tgt_enc["input_ids"].squeeze(0)
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

# Initializing Dataset and Dataloader
dataset = EnTaMarianDataset(df, tokenizer, max_length=128)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# ==========================================
# 3. Phase 1: Supervised Fine-Tuning (SFT)
# ==========================================
print("Starting Phase 1: Supervised Fine-Tuning...")
num_epochs = 1
optimizer = AdamW(model.parameters(), lr=3e-5)

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0.0
    # for batch in train_loader: ... (Standard loop you provided)
    # [Insert your training loop logic here]

print("Fine-tuning complete. Saving Phase 1 model...")
model.save_pretrained("marian_enhi_to_entamil_finetuned")

# ==========================================
# 4. Phase 2: Algorithm 1 (Self-Training)
# ==========================================

def run_self_training_adaptation(model, tokenizer, document_sentences, 
                                 passes=2, steps=3, alpha=1e-5, lam=0.01):
    """
    Implements Algorithm 1 from Mansimov et al. (2021).
    - theta_hat: The weights from Phase 1 (Original Weights).
    - l_pseudo: Cross-entropy using model's own predictions as targets.
    - lam: The decay prior strength (regularization).
    """
    print("\nStarting Phase 2: Document-Level Self-Training Adaptation...")
    
    # 1. Store the original weights (theta_hat) as a reference point
    # We clone them to ensure they stay fixed while 'model' updates.
    theta_hat = {n: p.clone().detach().to(device) for n, p in model.named_parameters()}
    
    # Process the document
    for p in range(1, passes + 1):
        print(f"Pass {p}/{passes}")
        
        for i, sentence in enumerate(document_sentences):
            # STEP A: Generate Pseudo-labels (Y_hat_i)
            model.eval()
            inputs = tokenizer(sentence, return_tensors="pt", padding=True).to(device)
            
            with torch.no_grad():
                # We use generate to get the model's best current guess
                pseudo_labels = model.generate(**inputs)
            
            model.train()
            
            # STEP B: Inner adaptation steps (j)
            for j in range(steps):
                # Forward pass using the model's own guess as the 'true' labels
                outputs = model(**inputs, labels=pseudo_labels)
                l_pseudo = outputs.loss
                
                # Backprop to find gradients
                model.zero_grad()
                l_pseudo.backward()
                
                # STEP C: Custom Weight Update with Decay Prior
                # Equation: θ ← θ - α∇θLpseudo + λ(θ_hat - θ)
                with torch.no_grad():
                    for name, param in model.named_parameters():
                        if param.grad is not None:
                            # Gradient descent part
                            grad_update = alpha * param.grad
                            # Decay prior part (rubber band back to theta_hat)
                            decay_pull = lam * (theta_hat[name] - param)
                            
                            # Apply combined update
                            param.data = param.data - grad_update + decay_pull
                
                model.zero_grad()

# ==========================================
# 5. Execution of Self-Training
# ==========================================

# Example: A 'Document' is a list of contextually related sentences.
target_document = [
    "Digital transformation is essential for modern business.",
    "It involves integrating technology into all areas of operation.",
    "This shift changes how companies deliver value to customers."
]

# Run the algorithm
run_self_training_adaptation(
    model=model,
    tokenizer=tokenizer,
    document_sentences=target_document,
    passes=2,   # p
    steps=3,    # j
    alpha=1e-5, # learning rate α
    lam=0.01    # decay prior λ
)

# ==========================================
# 6. Final Save
# ==========================================
print("Saving Final Self-Trained Model...")
model.save_pretrained("marian_final_adapted_document_model")
tokenizer.save_pretrained("marian_final_adapted_document_model")

In [ ]:
#hybrid loss 

In [ ]:
import pandas as pd
import torch
import copy
from torch.utils.data import Dataset, DataLoader
from transformers import MarianMTModel, MarianTokenizer, AdamW

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# 1. LOAD MODEL & TOKENIZER
# ==========================================
model_name = "Helsinki-NLP/opus-mt-en-hi" 
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)

# ==========================================
# 2. DATASET CLASS
# ==========================================
class EnTaMarianDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.en = df["en"].astype(str).tolist()
        self.ta = df["ta"].astype(str).tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.en)

    def __getitem__(self, idx):
        src_text = self.en[idx]
        tgt_text = self.ta[idx]

        src_enc = self.tokenizer(src_text, max_length=self.max_length, 
                                 truncation=True, padding="max_length", return_tensors="pt")
        
        with self.tokenizer.as_target_tokenizer():
            tgt_enc = self.tokenizer(tgt_text, max_length=self.max_length, 
                                     truncation=True, padding="max_length", return_tensors="pt")

        labels = tgt_enc["input_ids"].squeeze(0)
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": src_enc["input_ids"].squeeze(0),
            "attention_mask": src_enc["attention_mask"].squeeze(0),
            "labels": labels,
        }

# ==========================================
# 3. HYBRID LOSS & FREEZING LOGIC
# ==========================================

def prepare_model_for_hybrid_training(model):
    """
    Freezes all layers except the last two decoder layers 
    to preserve core linguistic knowledge.
    """
    # Freeze everything first
    for param in model.parameters():
        param.requires_grad = False
    
    # Unfreeze last two decoder layers
    # MarianMT: model.model.decoder.layers
    num_decoder_layers = len(model.model.decoder.layers)
    for i in range(num_decoder_layers - 2, num_decoder_layers):
        for param in model.model.decoder.layers[i].parameters():
            param.requires_grad = True
            
    # Unfreeze the final output head
    for param in model.lm_head.parameters():
        param.requires_grad = True
    
    return model

def calculate_hybrid_loss(model, batch, alpha=0.5):
    """
    L_hybrid = alpha * L_pseudo + (1 - alpha) * L_gold
    """
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    gold_labels = batch["labels"].to(device)

    # --- Part A: Gold Loss ---
    outputs_gold = model(input_ids=input_ids, attention_mask=attention_mask, labels=gold_labels)
    l_gold = outputs_gold.loss

    # --- Part B: Pseudo Loss ---
    model.eval()
    with torch.no_grad():
        # Generate model's own translation as target
        pseudo_labels = model.generate(input_ids=input_ids, attention_mask=attention_mask)
    model.train()

    # Calculate loss against the pseudo-labels
    outputs_pseudo = model(input_ids=input_ids, attention_mask=attention_mask, labels=pseudo_labels)
    l_pseudo = outputs_pseudo.loss

    # --- Weighted Sum ---
    return (alpha * l_pseudo) + ((1 - alpha) * l_gold)

# ==========================================
# 4. FULL EXECUTION SCRIPT
# ==========================================

# Load your dataframe
# df = pd.read_csv("pmindia_en_ta_39k.csv")
# dataset = EnTaMarianDataset(df, tokenizer)
# train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# A. Prepare model (Freezing)
model = prepare_model_for_hybrid_training(model)

# B. Setup Optimizer (only for parameters that require grad)
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=3e-5)

# C. Hybrid Training Loop
num_epochs = 1
alpha_param = 0.4 # Weight for pseudo-labels



print("Starting Hybrid Loss Training...")
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in train_loader:
        optimizer.zero_grad()
        
        # Compute Hybrid Loss
        loss = calculate_hybrid_loss(model, batch, alpha=alpha_param)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} | Hybrid Loss: {avg_loss:.4f}")

# ==========================================
# 5. SAVE FINAL MODEL
# ==========================================
model.save_pretrained("marian_hybrid_loss_model")
tokenizer.save_pretrained("marian_hybrid_loss_model")
print("Model saved successfully.")

In [1]:
#fine tuning 

In [ ]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    MarianMTModel, 
    MarianTokenizer, 
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer, 
    DataCollatorForSeq2Seq
)

# 1. Setup Device & Base Model
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "Helsinki-NLP/opus-mt-en-hi"  # The base MMT model
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)

# 2. Load & Prepare the PMIndia Dataset
# In the paper, they used 1277 aligned documents
# We assume your CSV 'pmindia_curated.csv' has columns: 'en' and 'hi'
df = pd.read_csv("pmindia_curated.csv")
raw_dataset = Dataset.from_pandas(df)

def preprocess_function(examples):
    inputs = [ex for ex in examples["en"]]
    targets = [ex for ex in examples["hi"]]
    
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the whole dataset
tokenized_dataset = raw_dataset.map(preprocess_function, batched=True)

# 3. Define Training Arguments (SFT Phase)
training_args = Seq2SeqTrainingArguments(
    output_dir="./marian_mmt_ft_results",
    evaluation_strategy="no",
    learning_rate=3e-5,       # Standard for fine-tuning
    per_device_train_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,       # Number of passes over the 1277 docs
    predict_with_generate=True,
    fp16=True if torch.cuda.is_available() else False, # Speed up if GPU
    push_to_hub=False,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 4. Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# 5. Start Supervised Fine-Tuning
print("Starting SFT (MMT + FT) phase...")
trainer.train()

# 6. Save the 'Robust Base'
# This model will now be used for Phase 4.3.2 and 4.3.3
model.save_pretrained("./marian_pmindia_base_ft")
tokenizer.save_pretrained("./marian_pmindia_base_ft")
print("Phase 4.3.4 Complete. Base model saved.")

In [2]:
#lora

In [ ]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    MarianMTModel, 
    MarianTokenizer, 
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer, 
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, TaskType

# 1. Load the "Robust Base" from Phase 4.3.4
device = "cuda" if torch.cuda.is_available() else "cpu"
model_path = "./marian_pmindia_base_ft" # The model saved in the previous step
tokenizer = MarianTokenizer.from_pretrained(model_path)
model = MarianMTModel.from_pretrained(model_path).to(device)

# 2. Configure LoRA (Low-Rank Adaptation)
# r: the rank (bottleneck size). lora_alpha: scaling factor.
# target_modules: In MarianMT, these are usually the attention layers.
lora_config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["q_proj", "v_proj"], # Adapting the Attention Query and Value
    lora_dropout=0.1,
    task_type=TaskType.SEQ_2_SEQ_LM
)

# 3. Apply LoRA to the model
# This freezes the base weights and adds the trainable adapters
model = get_peft_model(model, lora_config)

# Print trainable parameters to verify (usually <1% of total)
model.print_trainable_parameters()

# 

# 4. Prepare Dataset (Same as previous steps)
df = pd.read_csv("pmindia_curated.csv")
raw_dataset = Dataset.from_pandas(df)

def preprocess_function(examples):
    inputs = [ex for ex in examples["en"]]
    targets = [ex for ex in examples["hi"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = raw_dataset.map(preprocess_function, batched=True)

# 5. Define Training Arguments
# Note: LoRA usually requires a slightly higher learning rate than full FT
training_args = Seq2SeqTrainingArguments(
    output_dir="./marian_lora_results",
    learning_rate=1e-4, 
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True if torch.cuda.is_available() else False,
    logging_steps=100,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 6. Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# 7. Start LoRA Training
print("Starting LoRA Fine-Tuning...")
trainer.train()

# 8. Save the LoRA Adapters
# Note: This only saves the tiny adapter weights, not the whole model
model.save_pretrained("./marian_lora_adapters")
print("LoRA Fine-Tuning Complete.")

In [3]:
#span based context injection

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import MarianMTModel, MarianTokenizer, AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# 1. LOAD MODEL & ADD SPECIAL TOKENS
# ==========================================
model_name = "Helsinki-NLP/opus-mt-en-hi" 
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)

# Define the span tokens
special_tokens_dict = {'additional_special_tokens': ['<prev>', '</prev>', '<curr>', '</curr>', '<next>', '</next>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

# Resize the model embeddings to account for new tokens
model.resize_token_embeddings(len(tokenizer))

# ==========================================
# 2. CONTEXTUAL DATASET CLASS
# ==========================================
class SpanContextDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=256):
        """
        Expects a DataFrame where rows are in chronological order 
        belonging to the same document.
        """
        self.en_sentences = df["en"].astype(str).tolist()
        self.hi_sentences = df["hi"].astype(str).tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.en_sentences)

    def __getitem__(self, idx):
        # Identify neighbors for context injection
        prev_s = self.en_sentences[idx-1] if idx > 0 else ""
        curr_s = self.en_sentences[idx]
        next_s = self.en_sentences[idx+1] if idx < len(self.en_sentences)-1 else ""

        # Format input: <prev> Si-1 </prev> <curr> Si </curr> <next> Si+1 </next>
        contextual_input = (
            f"<prev> {prev_s} </prev> "
            f"<curr> {curr_s} </curr> "
            f"<next> {next_s} </next>"
        )
        
        # Target remains just the current sentence (Si) in Hindi
        target_text = self.hi_sentences[idx]

        # Tokenize
        src_enc = self.tokenizer(contextual_input, max_length=self.max_length, 
                                 truncation=True, padding="max_length", return_tensors="pt")
        
        with self.tokenizer.as_target_tokenizer():
            tgt_enc = self.tokenizer(target_text, max_length=128, 
                                     truncation=True, padding="max_length", return_tensors="pt")

        labels = tgt_enc["input_ids"].squeeze(0)
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": src_enc["input_ids"].squeeze(0),
            "attention_mask": src_enc["attention_mask"].squeeze(0),
            "labels": labels
        }

# ==========================================
# 3. HYBRID LOSS WITH SPAN CONTEXT
# ==========================================
def calculate_hybrid_span_loss(model, batch, alpha=0.4):
    """
    Combines the context-augmented input with the Hybrid Loss (αLpseudo + (1-α)Lgold)
    """
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    gold_labels = batch["labels"].to(device)

    # 1. Gold Loss (L_gold)
    outputs_gold = model(input_ids=input_ids, attention_mask=attention_mask, labels=gold_labels)
    l_gold = outputs_gold.loss

    # 2. Pseudo Loss (L_pseudo) using Context
    model.eval()
    with torch.no_grad():
        pseudo_labels = model.generate(input_ids=input_ids, attention_mask=attention_mask)
    model.train()
    
    outputs_pseudo = model(input_ids=input_ids, attention_mask=attention_mask, labels=pseudo_labels)
    l_pseudo = outputs_pseudo.loss

    return (alpha * l_pseudo) + ((1 - alpha) * l_gold)

# ==========================================
# 4. TRAINING EXECUTION
# ==========================================
# dataset = SpanContextDataset(df, tokenizer)
# loader = DataLoader(dataset, batch_size=8, shuffle=False) # Shuffle=False to keep doc order

optimizer = AdamW(model.parameters(), lr=2e-5)

print("Training MMT + FT + HL + Span...")
model.train()
# for batch in loader:
#     optimizer.zero_grad()
#     loss = calculate_hybrid_span_loss(model, batch)
#     loss.backward()
#     optimizer.step()

# Save final model
# model.save_pretrained("marian_span_context_model")

In [4]:
#bert based

In [ ]:
import torch
import torch.nn.functional as F
from bert_score import BERTScorer
from transformers import MarianMTModel, MarianTokenizer, AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# 1. LOAD MODEL & BERT SCORER
# ==========================================
model_name = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)

# Initialize BERTScorer for Semantic Loss (L_bert)
# Using 'bert-base-multilingual-cased' for English-Hindi/Tamil support
scorer = BERTScorer(lang="hi", device=device, rescale_with_baseline=True)

# ==========================================
# 2. HYBRID + BERTSCORE LOSS FUNCTION
# ==========================================

def compute_mmt_plus_all_loss(model, batch, alpha=0.4, lam_bert=0.3):
    """
    Implements Phase 4.3.7: MMT + FT + HL + Span + BERTScore-Loss
    """
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    gold_labels = batch["labels"].to(device) # Shape: [batch, seq_len]

    # --- 1. L_gold (Supervised Cross-Entropy) ---
    outputs_gold = model(input_ids=input_ids, attention_mask=attention_mask, labels=gold_labels)
    l_gold = outputs_gold.loss

    # --- 2. L_pseudo (Self-Training Cross-Entropy) ---
    model.eval()
    with torch.no_grad():
        # Generate model's own current best prediction
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask)
    model.train()
    
    # We must pad generated_ids to match gold_labels length or vice versa for the loss
    outputs_pseudo = model(input_ids=input_ids, attention_mask=attention_mask, labels=generated_ids)
    l_pseudo = outputs_pseudo.loss

    # --- 3. L_hybrid (Weighted Cross-Entropy) ---
    l_hybrid = (alpha * l_pseudo) + ((1 - alpha) * l_gold)

    # --- 4. L_bert (Semantic Loss) ---
    # Convert token IDs back to strings for BERTScore calculation
    # We use the 'generated_ids' vs 'gold_labels'
    decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    
    # Clean up gold labels (-100 to pad_id) before decoding
    clean_gold = gold_labels.clone()
    clean_gold[clean_gold == -100] = tokenizer.pad_token_id
    decoded_refs = tokenizer.batch_decode(clean_gold, skip_special_tokens=True)

    # BERTScore returns (P, R, F1). We use 1 - F1 as the semantic distance.
    _, _, f1 = scorer.score(decoded_preds, decoded_refs)
    l_bert = 1 - f1.mean()

    # --- 5. Total Loss Calculation ---
    l_total = ((1 - lam_bert) * l_hybrid) + (lam_bert * l_bert)
    
    return l_total

# ==========================================
# 3. TRAINING STEP EXAMPLE
# ==========================================
optimizer = AdamW(model.parameters(), lr=1e-5)

# Example batch processing
# for batch in dataloader:
#     optimizer.zero_grad()
#     loss = compute_mmt_plus_all_loss(model, batch, alpha=0.4, lam_bert=0.2)
#     loss.backward()
#     optimizer.step()